## Web Crawling, rottentomatoes 
- Selenium: crawling tool, BeautifulSoup: parsing 
- 대상: 
    - 전체 페이지 DVD&STREAMING MOIVEW(Browse All)  
    https://www.rottentomatoes.com/browse/in-theaters?minTomato=0&maxTomato=100&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=popularity
    - 1) 영화 제목
    - 2) 상영 날짜 
    - 3) 상세 페이지 링크 -> 다음 이동 경로
    
    - 영화 상세 페이지 
    - 1) 평점 및 평가 횟수(TOMATOMETER, AUDIENCE SCORE)
    - 2) MOIVE INFO
        - 1) 개요
        - 2) 장르
        - 3) 평점 => (전문가 평점 횟수/ 전체 평점 횟수) X 전문가 평점 = 비율이 고려된 가중 평균비율(전문가와 일반 모두에게 점수 높은 영화)
        - 4) 감독 
    - 3) CAST
        - 1) 배우 이름
    - 추가가능항목: 평가글, 비평가 등급별 리뷰 횟쉬
- 링크: https://movie.naver.com/movie/point/af/list.nhn


In [54]:
#  packagae download
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

### CH1. 전체 페이지
1) 영화 제목
2) 상세 페이지 링크 -> 해당 링크로 진입하여 크롤링      
3) 상영 가능일 

In [55]:
# Chorme driverload
driver_path =  'C:/Users/USER/hwao/chromedriver' 
url = 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=100&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release' 

browser = webdriver.Chrome(executable_path = driver_path)
browser.get(url)

* 주의) 
    1. page_source를 먼저 불어온 뒤, 작업해야함
    2. browser.quit()할 경우, 데이터를 불러올 수 없음 (항상 사이트가 켜져있는 상태에서 진행) 

In [56]:
time.sleep(3) # page loading 대기 

title = [] 
release_date = []
detail_page = []
browser.page_source

for i in range(35):
    browser.execute_script("return document.body.scrollHeight")
    time.sleep(3)
    browser.find_element_by_css_selector(".btn.btn-secondary-rt.mb-load-btn").click()
    



page_url_base = browser.find_elements_by_css_selector('div.movie_info')

for i in page_url_base:
    title.append(i.find_element_by_css_selector('div.movie_info  .movieTitle').text)
    release_date.append(i.find_element_by_css_selector('div.movie_info  .release-date').text)
    detail_page.append(i.find_element_by_css_selector('div.movie_info a').get_attribute('href'))

browser.quit()

In [57]:
print(f'제목 개수: {len(title)},발행 날짜 개수:{len(release_date)}, 상세페이지 링크 개수: {len(detail_page)}')

data_total_page = pd.DataFrame(title, columns = ["Title"]) # ,"release_date"])#,Directed_By)#, columns = ["Rating","Directed_By"])
data_total_page['Release_date'] = release_date
data_total_page['Detail_page'] = detail_page
data_total_page.to_csv("detail_page_link")

제목 개수: 1095,발행 날짜 개수:1095, 상세페이지 링크 개수: 1095


### 상세페이지 스크랩
    - 1) 청중 평가, 리뷰 횟수 
    - 2) 로튼 토마토 평가, 리뷰 횟수
    - => 청중+토마토 sum(평가)/ 리뷰 회숫 = 가중 평균
    - 3) 평가 단체
    - 4) 장르
    - 5) 감독 

In [58]:
# Chorme driverload
#driver_path =  'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 

time.sleep(5)

Tomatometer_per = []
Tomatometer_cnt = []
Audience_Score_per = []
Audience_Score_cnt = []
Rating = []
Genre = []
Directed_By = []
Release_Date = []
Movie_name = []

num = 0
for url in detail_page:
    browser = webdriver.Chrome(executable_path = driver_path)
    browser.get(url)
    browser.page_source
    
    num+=1
    print(f"진행상황:{(num/len(detail_page))*100}%")
    
    # 타이틀 
    try:
        Movie_name.append(browser.find_element_by_css_selector('.mop-ratings-wrap__title.mop-ratings-wrap__title--top').text)
    except:
        Movie_name.append(None)
        
    
    page_url_base = browser.find_elements_by_css_selector('.mop-ratings-wrap__info')

    
    # 점수

    for i in page_url_base:
        Tomatometer_per.append(i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(1)  .mop-ratings-wrap__percentage').text)
        Tomatometer_cnt.append(i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(1) .mop-ratings-wrap__review-totals:nth-of-type(1) small.mop-ratings-wrap__text--small').text)

        try:
            Audience_Score_per.append(i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(2) .mop-ratings-wrap__percentage').text)
        except:
            Audience_Score_per.append(None)
        try:
            temp =i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(2) .mop-ratings-wrap__review-totals:nth-of-type(1) strong.mop-ratings-wrap__text--small').text
            Audience_Score_cnt.append(temp[temp.find(":")+2:])
        except:
            #print(f"No element!")
            Audience_Score_cnt.append(None)

    #browser.quit()


    temp ={}
    try: 
        # Rating
        rating_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(1) div.meta-label.subtle')
        rating_label= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(1) div.meta-value')
        if rating_debug.text[:-1] == "Rating":
            Rating.append(rating_label.text)
        else:
            Rating.append(None) 
    
    except:
        Rating.append(None) 

    try:
        # Genre
        genre_label_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(2) .meta-label.subtle')
        genre_label= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(2) .meta-value')
        if genre_label_debug.text[:-1] == "Genre":
            Genre.append(genre_label.text)
        else:
            Genre.append(None) 

    except:
        Genre.append(None) 
        
    try:
        # Directed By
        directed_by_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(3) .meta-label.subtle')
        directed_by_label= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(3) .meta-value')
        if directed_by_debug.text[:-1] == "Directed By":
            Directed_By.append(directed_by_label.text)
        else:
            Directed_By.append(None) 
    except:
        Directed_By.append(None) 
    
    try:
        # Releas_Date
        release_date_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(6) .meta-label.subtle')
        release_date= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(6) .meta-value')
        
        if release_date_debug.text[:-1] == "On Disc/Streaming":
            Release_Date.append(release_date.text)
        else:
            Release_Date.append(None) 
    except:
        Release_Date.append(None) 
        
    browser.quit()



진행상황:0.091324200913242%
진행상황:0.182648401826484%
진행상황:0.273972602739726%
진행상황:0.365296803652968%
진행상황:0.45662100456621%
진행상황:0.547945205479452%
진행상황:0.6392694063926941%
진행상황:0.730593607305936%
진행상황:0.821917808219178%
진행상황:0.91324200913242%
진행상황:1.004566210045662%
진행상황:1.095890410958904%
진행상황:1.187214611872146%
진행상황:1.2785388127853883%
진행상황:1.36986301369863%
진행상황:1.461187214611872%
진행상황:1.552511415525114%
진행상황:1.643835616438356%
진행상황:1.7351598173515983%
진행상황:1.82648401826484%
진행상황:1.9178082191780823%
진행상황:2.009132420091324%
진행상황:2.1004566210045663%
진행상황:2.191780821917808%
진행상황:2.28310502283105%
진행상황:2.374429223744292%
진행상황:2.4657534246575343%
진행상황:2.5570776255707766%
진행상황:2.6484018264840183%
진행상황:2.73972602739726%
진행상황:2.8310502283105023%
진행상황:2.922374429223744%
진행상황:3.0136986301369864%
진행상황:3.105022831050228%
진행상황:3.1963470319634704%
진행상황:3.287671232876712%
진행상황:3.3789954337899544%
진행상황:3.4703196347031966%
진행상황:3.5616438356164384%
진행상황:3.65296803652968%
진행상황:3.744292237442922%
진행상황:3.83

진행상황:30.776255707762555%
진행상황:30.8675799086758%
진행상황:30.958904109589042%
진행상황:31.05022831050228%
진행상황:31.141552511415526%
진행상황:31.232876712328768%
진행상황:31.324200913242013%
진행상황:31.41552511415525%
진행상황:31.506849315068493%
진행상황:31.59817351598174%
진행상황:31.689497716894977%
진행상황:31.780821917808222%
진행상황:31.87214611872146%
진행상황:31.963470319634702%
진행상황:32.054794520547944%
진행상황:32.146118721461185%
진행상황:32.237442922374434%
진행상황:32.32876712328767%
진행상황:32.42009132420091%
진행상황:32.51141552511416%
진행상황:32.602739726027394%
진행상황:32.69406392694064%
진행상황:32.785388127853885%
진행상황:32.87671232876712%
진행상황:32.96803652968037%
진행상황:33.0593607305936%
진행상황:33.15068493150685%
진행상황:33.242009132420094%
진행상황:33.33333333333333%
진행상황:33.42465753424658%
진행상황:33.51598173515982%
진행상황:33.60730593607306%
진행상황:33.6986301369863%
진행상황:33.789954337899545%
진행상황:33.881278538812786%
진행상황:33.97260273972603%
진행상황:34.06392694063927%
진행상황:34.15525114155251%
진행상황:34.24657534246575%
진행상황:34.337899543378995%
진행상황:34.42922374429224%
진

진행상황:61.7351598173516%
진행상황:61.826484018264836%
진행상황:61.917808219178085%
진행상황:62.009132420091326%
진행상황:62.10045662100456%
진행상황:62.19178082191781%
진행상황:62.28310502283105%
진행상황:62.374429223744286%
진행상황:62.465753424657535%
진행상황:62.55707762557078%
진행상황:62.648401826484026%
진행상황:62.73972602739726%
진행상황:62.8310502283105%
진행상황:62.92237442922375%
진행상황:63.013698630136986%
진행상황:63.10502283105023%
진행상황:63.19634703196348%
진행상황:63.287671232876704%
진행상황:63.37899543378995%
진행상황:63.4703196347032%
진행상황:63.561643835616444%
진행상황:63.65296803652968%
진행상황:63.74429223744292%
진행상황:63.83561643835617%
진행상황:63.926940639269404%
진행상황:64.01826484018265%
진행상황:64.10958904109589%
진행상황:64.20091324200912%
진행상황:64.29223744292237%
진행상황:64.38356164383562%
진행상황:64.47488584474887%
진행상황:64.5662100456621%
진행상황:64.65753424657534%
진행상황:64.74885844748859%
진행상황:64.84018264840182%
진행상황:64.93150684931507%
진행상황:65.02283105022832%
진행상황:65.11415525114155%
진행상황:65.20547945205479%
진행상황:65.29680365296804%
진행상황:65.38812785388129%
진행상황:65.47

진행상황:93.15068493150685%
진행상황:93.2420091324201%
진행상황:93.33333333333333%
진행상황:93.42465753424658%
진행상황:93.51598173515981%
진행상황:93.60730593607306%
진행상황:93.69863013698631%
진행상황:93.78995433789954%
진행상황:93.88127853881278%
진행상황:93.97260273972603%
진행상황:94.06392694063926%
진행상황:94.15525114155251%
진행상황:94.24657534246576%
진행상황:94.337899543379%
진행상황:94.42922374429223%
진행상황:94.52054794520548%
진행상황:94.61187214611873%
진행상황:94.70319634703196%
진행상황:94.79452054794521%
진행상황:94.88584474885845%
진행상황:94.97716894977168%
진행상황:95.06849315068493%
진행상황:95.15981735159818%
진행상황:95.25114155251143%
진행상황:95.34246575342465%
진행상황:95.4337899543379%
진행상황:95.52511415525115%
진행상황:95.61643835616438%
진행상황:95.70776255707763%
진행상황:95.79908675799086%
진행상황:95.8904109589041%
진행상황:95.98173515981735%
진행상황:96.0730593607306%
진행상황:96.16438356164385%
진행상황:96.25570776255708%
진행상황:96.34703196347031%
진행상황:96.43835616438356%
진행상황:96.5296803652968%
진행상황:96.62100456621005%
진행상황:96.7123287671233%
진행상황:96.80365296803653%
진행상황:96.89497716894977%


In [75]:
data = pd.DataFrame(Movie_name[:1093], columns = ["Title"]) # ,"release_date"])#,Directed_By)#, columns = ["Rating","Directed_By"])
data['Release_date'] = Release_Date[:1093]
data['Genre'] = Genre[:1093]

data['Tometer_per'] = Tomatometer_per
data['Tomatometer_cnt'] = Tomatometer_cnt
data['Audience_Score_per'] = Audience_Score_per
data['Audience_Score_cnt'] = Audience_Score_cnt

data['Rating_Compony'] = Rating[:1093]
data['Directed_By'] = Directed_By[:1093]


#data['detail_page'] = detail_page[:1093]
len(Rating)

## 점수 칸만 1095개 나머지는 1093개 
data

,Title,Release_date,Genre,Tometer_per,Tomatometer_cnt,Audience_Score_per,Audience_Score_cnt,Rating_Compony,Directed_By
0,GAME OF DEATH,"Jul 14, 2020","Horror, Mystery & Suspense",58%,26,26%,52,NR,"Sébastien Landry, Laurence Morais-Lagace"
1,RETALIATION (ROMANS),"Jul 24, 2020","Drama, Mystery & Suspense",86%,14,None,None,"R (for disturbing violent/sexual content, lang...","Ludwig Shammasian, Paul Shammasian"
2,FISHERMAN'S FRIENDS,None,"Comedy, Drama, Musical & Performing Arts",65%,46,72%,111,"PG-13 (for some strong language, and suggestiv...",Chris Foggin
3,THE SUNLIT NIGHT,None,"Drama, Romance",34%,47,None,None,NR,David Wnendt
4,SKYMAN,"Jul 7, 2020","Documentary, Drama, Science Fiction & Fantasy",63%,16,None,None,NR,Daniel Myrick
5,BABYSPLITTERS,None,Comedy,71%,14,94%,34,NR,Sam Friedlander
6,"YES, GOD, YES","Jul 28, 2020",Drama,92%,75,91%,121,R (for sexual content and some nudity),Karen Maine
7,SAMURAI MARATHON,None,"Action & Adventure, Drama",75%,8,None,None,NR,Bernard Rose
8,GUEST OF HONOUR,"Jul 10, 2020",Drama,37%,57,None,None,NR,Atom Egoyan
9,BLACK MAGIC FOR WHITE BOYS,"Jul 3, 2020",Comedy,80%,10,None,None,NR,Onur Tukel


In [76]:
data

,Title,Release_date,Genre,Tometer_per,Tomatometer_cnt,Audience_Score_per,Audience_Score_cnt,Rating_Compony,Directed_By
0,GAME OF DEATH,"Jul 14, 2020","Horror, Mystery & Suspense",58%,26,26%,52,NR,"Sébastien Landry, Laurence Morais-Lagace"
1,RETALIATION (ROMANS),"Jul 24, 2020","Drama, Mystery & Suspense",86%,14,None,None,"R (for disturbing violent/sexual content, lang...","Ludwig Shammasian, Paul Shammasian"
2,FISHERMAN'S FRIENDS,None,"Comedy, Drama, Musical & Performing Arts",65%,46,72%,111,"PG-13 (for some strong language, and suggestiv...",Chris Foggin
3,THE SUNLIT NIGHT,None,"Drama, Romance",34%,47,None,None,NR,David Wnendt
4,SKYMAN,"Jul 7, 2020","Documentary, Drama, Science Fiction & Fantasy",63%,16,None,None,NR,Daniel Myrick
5,BABYSPLITTERS,None,Comedy,71%,14,94%,34,NR,Sam Friedlander
6,"YES, GOD, YES","Jul 28, 2020",Drama,92%,75,91%,121,R (for sexual content and some nudity),Karen Maine
7,SAMURAI MARATHON,None,"Action & Adventure, Drama",75%,8,None,None,NR,Bernard Rose
8,GUEST OF HONOUR,"Jul 10, 2020",Drama,37%,57,None,None,NR,Atom Egoyan
9,BLACK MAGIC FOR WHITE BOYS,"Jul 3, 2020",Comedy,80%,10,None,None,NR,Onur Tukel


In [77]:
data.to_csv("data_rating")

In [78]:
# detail_page + "/reviews" = review page
time.sleep(3)

Title = []
Review = [] 
Reviewer = []
Review_date = []

num = 0
for page in detail_page:
    temp_url = page+"/reviews"
    browser = webdriver.Chrome(executable_path = driver_path)
    browser.get(temp_url)
    
    num+=1
    print(f"진행상황:{(num/len(detail_page))*100}%")

    try: 
        ###### 예외 처리 통과 구문
        browser.find_element_by_css_selector('div.center   .unstyled.articleLink')
        browser.find_elements_by_css_selector('div.the_review')
        browser.find_elements_by_css_selector('div.row.review_table_row a.unstyled.bold.articleLink')
        browser.find_elements_by_css_selector('div.review_area .review-date.subtle.small')
        ##########################
        
        name = browser.find_element_by_css_selector('div.center   .unstyled.articleLink').text


        reivew_boxes = browser.find_elements_by_css_selector('div.the_review')
        for review in reivew_boxes:
            Review.append(review.text)
            Title.append(name) # name 리뷰 개수만큼 넣어주기 

        reviewer_boxes = browser.find_elements_by_css_selector('div.row.review_table_row a.unstyled.bold.articleLink')
        for reviewer in reviewer_boxes:
            Reviewer.append(reviewer.text)

        review_date_boxes = browser.find_elements_by_css_selector('div.review_area .review-date.subtle.small')
        for reviewe_date in review_date_boxes:
            Review_date.append(reviewe_date.text)
        browser.quit()
    except:
        continue

진행상황:0.091324200913242%
진행상황:0.182648401826484%
진행상황:0.273972602739726%
진행상황:0.365296803652968%
진행상황:0.45662100456621%
진행상황:0.547945205479452%
진행상황:0.6392694063926941%
진행상황:0.730593607305936%
진행상황:0.821917808219178%
진행상황:0.91324200913242%
진행상황:1.004566210045662%
진행상황:1.095890410958904%
진행상황:1.187214611872146%
진행상황:1.2785388127853883%
진행상황:1.36986301369863%
진행상황:1.461187214611872%
진행상황:1.552511415525114%
진행상황:1.643835616438356%
진행상황:1.7351598173515983%
진행상황:1.82648401826484%
진행상황:1.9178082191780823%
진행상황:2.009132420091324%
진행상황:2.1004566210045663%
진행상황:2.191780821917808%
진행상황:2.28310502283105%
진행상황:2.374429223744292%
진행상황:2.4657534246575343%
진행상황:2.5570776255707766%
진행상황:2.6484018264840183%
진행상황:2.73972602739726%
진행상황:2.8310502283105023%
진행상황:2.922374429223744%
진행상황:3.0136986301369864%
진행상황:3.105022831050228%
진행상황:3.1963470319634704%
진행상황:3.287671232876712%
진행상황:3.3789954337899544%
진행상황:3.4703196347031966%
진행상황:3.5616438356164384%
진행상황:3.65296803652968%
진행상황:3.744292237442922%
진행상황:3.83

진행상황:30.776255707762555%
진행상황:30.8675799086758%
진행상황:30.958904109589042%
진행상황:31.05022831050228%
진행상황:31.141552511415526%
진행상황:31.232876712328768%
진행상황:31.324200913242013%
진행상황:31.41552511415525%
진행상황:31.506849315068493%
진행상황:31.59817351598174%
진행상황:31.689497716894977%
진행상황:31.780821917808222%
진행상황:31.87214611872146%
진행상황:31.963470319634702%
진행상황:32.054794520547944%
진행상황:32.146118721461185%
진행상황:32.237442922374434%
진행상황:32.32876712328767%
진행상황:32.42009132420091%
진행상황:32.51141552511416%
진행상황:32.602739726027394%
진행상황:32.69406392694064%
진행상황:32.785388127853885%
진행상황:32.87671232876712%
진행상황:32.96803652968037%
진행상황:33.0593607305936%
진행상황:33.15068493150685%
진행상황:33.242009132420094%
진행상황:33.33333333333333%
진행상황:33.42465753424658%
진행상황:33.51598173515982%
진행상황:33.60730593607306%
진행상황:33.6986301369863%
진행상황:33.789954337899545%
진행상황:33.881278538812786%
진행상황:33.97260273972603%
진행상황:34.06392694063927%
진행상황:34.15525114155251%
진행상황:34.24657534246575%
진행상황:34.337899543378995%
진행상황:34.42922374429224%
진

진행상황:61.7351598173516%
진행상황:61.826484018264836%
진행상황:61.917808219178085%
진행상황:62.009132420091326%
진행상황:62.10045662100456%
진행상황:62.19178082191781%
진행상황:62.28310502283105%
진행상황:62.374429223744286%
진행상황:62.465753424657535%
진행상황:62.55707762557078%
진행상황:62.648401826484026%
진행상황:62.73972602739726%
진행상황:62.8310502283105%
진행상황:62.92237442922375%
진행상황:63.013698630136986%
진행상황:63.10502283105023%
진행상황:63.19634703196348%
진행상황:63.287671232876704%
진행상황:63.37899543378995%
진행상황:63.4703196347032%
진행상황:63.561643835616444%
진행상황:63.65296803652968%
진행상황:63.74429223744292%
진행상황:63.83561643835617%
진행상황:63.926940639269404%
진행상황:64.01826484018265%
진행상황:64.10958904109589%
진행상황:64.20091324200912%
진행상황:64.29223744292237%
진행상황:64.38356164383562%
진행상황:64.47488584474887%
진행상황:64.5662100456621%
진행상황:64.65753424657534%
진행상황:64.74885844748859%
진행상황:64.84018264840182%
진행상황:64.93150684931507%
진행상황:65.02283105022832%
진행상황:65.11415525114155%
진행상황:65.20547945205479%
진행상황:65.29680365296804%
진행상황:65.38812785388129%
진행상황:65.47

진행상황:93.15068493150685%
진행상황:93.2420091324201%
진행상황:93.33333333333333%
진행상황:93.42465753424658%
진행상황:93.51598173515981%
진행상황:93.60730593607306%
진행상황:93.69863013698631%
진행상황:93.78995433789954%
진행상황:93.88127853881278%
진행상황:93.97260273972603%
진행상황:94.06392694063926%
진행상황:94.15525114155251%
진행상황:94.24657534246576%
진행상황:94.337899543379%
진행상황:94.42922374429223%
진행상황:94.52054794520548%
진행상황:94.61187214611873%
진행상황:94.70319634703196%
진행상황:94.79452054794521%
진행상황:94.88584474885845%
진행상황:94.97716894977168%
진행상황:95.06849315068493%
진행상황:95.15981735159818%
진행상황:95.25114155251143%
진행상황:95.34246575342465%
진행상황:95.4337899543379%
진행상황:95.52511415525115%
진행상황:95.61643835616438%
진행상황:95.70776255707763%
진행상황:95.79908675799086%
진행상황:95.8904109589041%
진행상황:95.98173515981735%
진행상황:96.0730593607306%
진행상황:96.16438356164385%
진행상황:96.25570776255708%
진행상황:96.34703196347031%
진행상황:96.43835616438356%
진행상황:96.5296803652968%
진행상황:96.62100456621005%
진행상황:96.7123287671233%
진행상황:96.80365296803653%
진행상황:96.89497716894977%


In [79]:
review_data = pd.DataFrame(Title,columns= ["Title"])
review_data['Review_date'] = Review_date
review_data["Reviewer"] = Reviewer
review_data["Review"] = Review
review_data

ValueError: Length of values does not match length of index

In [ ]:
review_data.to_csv("Reviw_data")

#### 기타: 리뷰 페이지 URL 이동(작업중)

In [ ]:
temp_url = "https://www.rottentomatoes.com/m/the_prince_2020"
browser = webdriver.Chrome(executable_path = driver_path)
browser.get(temp_url)

num = 2
while True :
    try:
        print(temp)
        temp = temp_url + "?type=&sort=&page=" + str(num)
        browser.execute_script("return document.body.scrollHeight")
        time.sleep(3)
        browser.find_element_by_css_selector("a.btn btn-xs.btn-primary-rt").click()
        num +=1
        print(temp)
    except:
        print("end page")
        break
    

In [ ]:
len()